In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
import pprint
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from collections import OrderedDict

from dataset import *
from runner import ExptRunner
from networks.imageencoder import *
from networks.imagedecoder import *
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.dense import *
from networks.lossfunctions import *
from networks.multinet import *
from networks.special import *
from helpers import *
import utils
import simplereacherdimensions
import simplereacheradapters

In [3]:
import warnings
warnings.filterwarnings("ignore", category=torch.serialization.SourceChangeWarning)

In [4]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [5]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [6]:
if torch.cuda.is_available():
    print (torch.cuda.current_device())
    torch.cuda.set_device(device)
    print (torch.cuda.current_device())

In [7]:
root_folder = 'data/simple_reacher/training'
# number_of_trajectories = 10
number_of_trajectories = 5000

In [8]:
start = time.time()
train_data = SimpleReacherOnDemandDataset(root_folder, range(4750), device=device)
test_data = SimpleReacherOnDemandDataset(root_folder, range(4750, 5000), device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 34.68616437911987 sec


In [9]:
print (len(train_data))
print (len(test_data))

98785
5283


In [10]:
def run_expts(train_epochs):
    for i in range(len(expts)):
        arg_lists = expts[i]
        cons_args = arg_lists[0]
        train_args = {}
        if (len(arg_lists)) > 1:
            train_args = arg_lists[1]
        test_args = {}
        if (len(arg_lists)) > 2:
            test_args = arg_lists[2]
        
        runner = ExptRunner(train_data=train_data, test_data=test_data, device=device, **cons_args)
        print ("Experiment logs folder: {}".format(runner.log_folder))
        with open(runner.log_folder + '/expt.txt', 'w') as f:
            f.write(pprint.pformat(arg_lists))
        runner.train(train_epochs, **train_args)
        runner.test(**test_args)

In [13]:
expts = [
    [{
        "expt_prefix":'ImageAutoEncoder-Reacher-ComposedAutoEncoder-MSE',
        "net_func": lambda: ComposedAutoEncoder(x_dim=3, z_dim=4096, img_res=256,layers_channels=[16,16,16,16], useMaxPool=True, device=device),
        "data_adapter_func": simplereacheradapters.It_scaled_adapter,
        "loss_adapter_func": mse_loss_adapter,
     },
    {
    },
    {
        "loss_adapter": l1_loss_adapter
    }],
]

In [ ]:
train_epochs = 2
run_expts(train_epochs)

Experiment logs folder: runs/07-18-14-52-15-ImageAutoEncoder-Reacher-ComposedAutoEncoder-MSE


In [26]:
net = ComposedAutoEncoder(x_dim=3, z_dim=64, img_res=256,layers_channels=[16,16,16,16], useMaxPool=True, device=device)

In [19]:
%reset -f ImageEncoder

Don't know how to reset  imageencoder, please run `%reset?` for details


In [23]:
ip_batch = torch.rand(10, 256*256)

In [24]:
ip_batch.shape

torch.Size([10, 65536])

In [27]:
op_batch = net(ip_batch)

In [28]:
op_batch.shape

torch.Size([10, 65536])

In [22]:
net

ComposedAutoEncoder(
  (encoder): ImageEncoder(
    (net): Sequential(
      (enc_conv0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (enc_relu0): ReLU()
      (enc_maxpool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (enc_conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (enc_relu1): ReLU()
      (enc_maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (enc_conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (enc_relu2): ReLU()
      (enc_maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (enc_conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (enc_relu3): ReLU()
      (enc_maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (decoder): ImageDecoder(
    (net): Sequential(
      (dec_convt0): ConvTranspose2d(16